In [ ]:
import apache_beam as beam
import json
import logging
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions

In [ ]:
project_id = ""
dataset_id = ""
table_id = ""
subscription_id =""

In [ ]:
"""
Clean cache
rm -rf ~/.apache_beam
"""

In [ ]:
import apache_beam as beam
import json
import logging
import sys
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions


logging.getLogger().setLevel(logging.INFO)


sys.argv = sys.argv[:1]

def decode_message(msg):
    
    output = msg.decode('utf-8')
    logging.info("New PubSub Message: %s", output)
    
    return json.loads(output)

pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True

with beam.Pipeline(options=pipeline_options) as p:
    (
        p 
            | "ReadFromPubSub" >> beam.io.ReadFromPubSub(subscription=f'projects/{project_id}/subscriptions/{subscription_id}')
            | "Decode msg" >> beam.Map(decode_message)
            | "Write to BigQuery" >> beam.io.WriteToBigQuery(
                table=f"{project_id}:{dataset_id}.{table_id}",
                schema='temp:INTEGER', 
                create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
            )
    )